# Ocenianie modelu regresji

Ciąg dalszy ćwiczeń z poprzedniego laboratorium.

## 1. Testy diagnostyczne reszt

Aby model regresji liniowej był prawidłowy, reszty (różnice między wartościami przewidywanymi przez model a rzeczywistymi) muszą spełniać kilka kryteriów:

- ich średnia wynosi zero,

In [ ]:
mean(rowery_model2$residuals) # średnia jest bardzo bliska 0, więc kryterium jest spełnione

- mają rozkład normalny,

In [ ]:
library(olsrr)

ols_plot_resid_hist(rowery_model2)
ols_plot_resid_qq(rowery_model2)

- homoskedastyczność (wartości zmiennej niezależnej mają taką samą wariancję)

In [ ]:
ols_plot_resid_fit(rowery_model2)

- nie są skorelowane

In [ ]:
library(car)
# Test Durbina-Watsona
# 0 < DW < 2: dodatnia autokorelacja
# DW = 2: zerowa autokorelacja
# 2 < DW < 4: autokorelacja ujemna

durbinWatsonTest(rowery_model2)

## 2. Analiza punktów wpływowych

Punkty wpływowe, to skrajne wartości zmiennych predyktorów, które mają znaczący wpływ na model regresji. W przypadku regresji wielokrotnej trudno jest znaleźć wartości odstające, które identyfikują punkty wpływowe. W tym celu stosuje się test statystyczny _odległość Cooka_.

In [ ]:
library(olsrr)
# Wykres odległości Cooka
ols_plot_cooksd_chart(rowery_model2)

# Wyświetlenie wartości odstających według malejącej odległości Cooka
cooks_outliers <- ols_plot_cooksd_chart(rowery_model2, print_plot = FALSE)$outliers
arrange(cooks_outliers, desc(cooks_distance))

### Zadanie: 

1. Porównaj obserwację o najwyższej odległości Cooka z podsumowaniem statystycznym pozostałych danych.
2. Zapisz indeksy 25 zidentyfikowanych punktów wpływowych w zmiennej `outlier_index`.
3. Porównaj podsumowania statystyczne zidentyfikowanych punktów wpływowych z podsumowaniem reszty danych. Jakie obserwujesz różnice?
4. Porównaj rozkład statystyczny oryginalnych danych z rozkładem statystycznym danych bez elementów odstających. Na tej podstawie zdecyduj, czy można usunąć z danych elementy odstające. Jeśli tak - zapisz kopię zbioru danych bez elementów odstających jako `rowery2`.

# Tu wpisz swoje rozwiązanie

## 3. Współliniowość

Zjawisko współliniowości zachodzi wówczas, gdy zmienne predyktorów (dwie lub więcej) są ze sobą mocno skorelowane. W modelach regresji liniowej skutkuje to zawyżonymi błędami standardowymi i utrudnia oddzielenie wpływu poszczególnych predyktorów na odpowiedź.

Do wykrywania współliniowości służy współczynnik inflacji wariancji VIF (ang. _Variance Inflation Factor_). $VIF = \frac{1}{1 - R^2_k = \frac{1}{Tolerance}}$, gdzie $R^2_k$ to współczynnik determinacji $R^2$ równania regresji, gdzie predyktor $k$ znajduje się po lewej stronie, a wszystkie pozostałe zmienne predyktorów po prawej.

Przyjmuje się, że współliniowość istnieje wówczas, gdy $VIF > 5$ ($Tolerance < 0.2$).

Oblicz współczynnik VIF dla zmiennych predyktorów modelu `rowery_model2`.

In [ ]:
ols_vif_tol(rowery_model2)

## 4. Ulepszanie modelu

### Uwzględnienie relacji nieliniowych - regresja wielomianowa

Wykres zależności zmiennej `rentals` od pozostałych predyktorów. Niebieska linia - dopasowanie regresji liniowej, czerwona - dopasowanie po wprowadzeniu predyktorów wielomianowych.

In [ ]:
#library(ggplot2)

#df <- data.frame(x,y)
humid <- ggplot(data=rowery2, aes(x=humidity, y=rentals)) + geom_point() + 
  geom_smooth(method = 'lm', se = TRUE) +
  geom_smooth(method = 'gam', colour = 'red')
wind <-  ggplot(data=rowery2, aes(x=windspeed, y=rentals)) + geom_point() + 
  geom_smooth(method = 'lm', se = TRUE) +
  geom_smooth(method = 'gam', colour = 'red')
temp <-  ggplot(data=rowery2, aes(x=temperature, y=rentals)) + geom_point() + 
  geom_smooth(method = 'lm', se = TRUE) +
  geom_smooth(method = 'gam', colour = 'red')

library(gridExtra)
grid.arrange(humid, wind, temp, nrow=1, ncol=3)

Dodaj do modelu wersje predyktorów podniesionych do kwadratu:

In [ ]:
# Dodanie nowych predyktorów
rowery2 <- rowery2 %>%
  mutate(humidity2 = humidity^2) %>%
  mutate(windspeed2 = windspeed^2) %>%
  mutate(temperature2 = temperature^2)

# Utworzenie nowego modelu liniowego
rowery_model3 <- lm(data = rowery2,
                    rentals ~ humidity + windspeed + temperature +
                      humidity2 + windspeed2 + temperature2)

summary(rowery_model3)

Sprawdź, czy wszystkie predyktory są istotne. Jeśli nie - popraw model usuwając zmienne nieistotne. Porównaj dane diagnostyczne nowego modelu z danymi modelu poprzedniego. Co się zmieniło?

### Uwzględnienie zmiennych kategorialnych

Za pomocą funkcji `summary()` wyświetl podsumowanie dla zmiennych czynnikowych `season`, `holiday`, `weekday` oraz `weather`.

Zamień wartości liczbowe reprezentujące wartości kategorialne na opisowe. Służy do tego funkcja `revalue()` z pakiety `plyr`.

In [ ]:
rowery2 <- rowery2 %>%
  mutate(season = plyr::revalue(season, c("1" = "Zima", "2" = "Wiosna",
                                    "3" = "Lato", "4" = "Jesien"))) %>%
  mutate(holiday = plyr::revalue(holiday, c("0" = "Nie", "1" = "Tak"))) %>%
  mutate(weekday = plyr::revalue(weekday, c("0" = "Niedziela","1" = "Poniedzialek", 
                                      "2" = "Wtorek", "3" = "Sroda", "4" = "Czwartek",
                                      "5" = "Piatek", "6" = "Sobota"))) %>%
  mutate(weather = plyr::revalue(weather, c("1" = "Ladna pogoda", "2" = "Lekkie opady",
                                      "3" = "Obfite opady")))

Utwórz kolejny model, uwzględniający wybraną zmienną kategorialną. Przeanalizuj podsumowanie modelu.

### Interakcje między zmiennymi

Zdarza się, że dwie zmienne mają łączny wpływ na odpowiedź. Na przykład interakcje mogą zachodzić między zmiennymi `weather` oraz `windspeed` lub między zmiennymi `weather` a `temperature`. Efekt interakcji można uwzględnić w modelu regresji przy użyciu operatora `*`. 

Utwórz nowy model regresji, uwzględniający interakcję `weather * windspeed`.

### Wybieranie ważnych zmiennych

W zbiorze danych `rowery2` dodaj trzy predyktory oparte na zmiennej `date`: 

- `dzien` - liczba dni, jakie upłynęły od dnia rozpoczęcia programu (różnica między zmienną `date` a minimalną wartością zmiennej `date`)

- `miesiac`

- `rok`.

In [ ]:
library(lubridate)

rowery2 <- rowery2 %>%
  mutate(dzien = as.numeric(date - min(date))) %>%
  mutate(miesiac = as.factor(month(date))) %>%
  mutate(rok = as.factor(year(date))) %>%
  select(-date)

Zastosuj funkcję `ols_step_both_p()` do selekcji zmiennych:

In [ ]:
ols_step_both_p(
  model = lm(
    data = rowery2,
    rentals ~ humidity + weekday + holiday + 
      temperature + humidity2 + temperature2 + season +
      windspeed * weather + realfeel + dzien + miesiac + rok
  ),
  pent = 0.2, # wartość progowa p zmiennych uwzględnianych w modelu
  prem = 0.01, # progowa wartość p zmiennych usuwanych z modelu
  details = FALSE # flaga wskazująca jak dużo informacji wyświetlać
)

Utwórz nowy model, wykorzystując zmienne wybrane podczas selekcji. Następnie ponownie sprawdź homoskedastyczność modelu.

# Tu wpisz swoje rozwiązanie